In [1]:

import os, sys

home="/Users/garmu"
proj_path = home+"/Dropbox/asdc/web/"
# This is so Django knows where to find stuff.
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mysite.settings")
sys.path.append(proj_path)

# This is so my local_settings.py gets loaded.
os.chdir(proj_path)

# This is so models get loaded.
from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()


from asim.models import * 
    
def writedicttocdf(cdffilename,mytgfobs2,czt_det_counts  , bgo_det_counts):

    from spacepy import pycdf
    import os 

    pycdf.lib.set_backward(False)

    #cdffilename='/Users/garmu/Dropbox/notebooks/asimdata/testmxgsdata.cdf'
    try:
        os.remove(cdffilename) 
    except:
        print (cdffilename, "does not exist ")
    bad_keys=['_state', 'detector_counts','mxgs_trig_count']
    bad_keys=['_state', 'detector_counts']

    with pycdf.CDF(cdffilename, '') as cdf_file:    
        for key in sorted(mytgfobs2.__dict__.iterkeys()):
            if key not in bad_keys:
                value=mytgfobs2.__dict__[key]
                #print (key, value)
                cdf_file.new(key, value, recVary=False)
                #cdf_file.attrs[key] = value

        for key in sorted(czt_det_counts.iterkeys()):
            value=czt_det_counts[key]
            cdf_file.new(key, value, recVary=False)

        for key in sorted(bgo_det_counts.iterkeys()):
            value=bgo_det_counts[key]
            #print (key)
            cdf_file.new(key, value, recVary=False)
            #cdf_file.attrs['mxgs_trig_count'] = mytgfobs2.__dict__['mxgs_trig_count']
            #cdf_file.attrs[key] = value
    print ('wrote to file' , cdffilename)

def testfiletodict(fname):

    with open(fname, 'r') as fileData:
        TGFdict = {}
        tag = None
        tag2= None
        for line in fileData:
            #line = line.strip()
            #print (line)
            if not line:
                continue
            if line[0] == ' ':
                tag = line[1:].strip()
                tag2=tag.replace(' ','_')
                tag2=tag2.replace('-','_')
                tag2=tag2.lower()
                TGFdict[tag2] = ''
            else:
                assert tag is not None, 'Invalid format, does not have space at beginning of tag line'
                TGFdict[tag2] += line
    return TGFdict


def convert_detector_counts (TGFdict):
    '''
    trim the first four hex characters /zeros (16 bits)
    add 0x before?
    then convert to bitstring types and check whether from CZT or BGO
    then extract 12 bits pulse height
    20 bits timestamp

    '''

    detector_counts=TGFdict['detector_counts'].splitlines()

    print (detector_counts[1])

    from bitstring import BitArray, BitStream

    def remove_cruft(s):
        return BitArray('0x'+s[4:])


    det_counts=[remove_cruft(s) for s in detector_counts]

    czt_dau_address=[]
    czt_asic_address=[]
    czt_asic_channel=[]
    czt_pulse_height=[]
    czt_multi_hit=[]
    czt_timestamp=[]

    czt_det_counts = {'czt_dau_address': czt_dau_address, \
                      'czt_asic_address': czt_asic_address, \
                      'czt_asic_channel': czt_asic_channel, \
                      'czt_pulse_height': czt_pulse_height, \
                      'czt_multi_hit': czt_multi_hit, \
                      'czt_timestamp': czt_timestamp     }



    bgo_address=[]
    bgo_fast=[]
    bgo_ovf=[]
    bgo_valley=[]
    bgo_pulse_height=[]
    bgo_timestamp=[]
    bgo_fast_timestamp=[]

    bgo_det_counts = {'bgo_address': bgo_address, \
                      'bgo_ovf': bgo_ovf, \
                      'bgo_fast': bgo_fast, \
                      'bgo_valley': bgo_valley, \
                      'bgo_pulse_height': bgo_pulse_height, \
                      'bgo_timestamp': bgo_timestamp, \
                      'bgo_fast_timestamp': bgo_fast_timestamp     }

    for i in range (0,len(det_counts)):
        a=det_counts[i]
        #print (a[0:2])
        if (a[0:2]== '0b00') :
            flagbits= a[0:2].uint
            dau_address=a[2:4].uint
            asic_address = a[4:9].uint
            asic_channel = a[9:16].uint
            pulse_height = a[16:26].uint
            multi_hit = a[26:28].uint
            timestamp = a[28:48].uint  

            czt_det_counts['czt_dau_address'].append(dau_address)
            czt_det_counts['czt_asic_address'].append(asic_address)
            czt_det_counts['czt_asic_channel'].append(asic_channel)
            czt_det_counts['czt_pulse_height'].append(pulse_height)
            czt_det_counts['czt_multi_hit'].append(multi_hit)
            czt_det_counts['czt_timestamp'].append(timestamp)
            """x
            print 'CZT' ,flagbits  ,\
                   'dau_add' ,dau_address, \
                   'asic_add' ,asic_address, \
                   'asic_chan' ,asic_channel, \
                   'puls_ht',pulse_height,\
                   'multi_hit', multi_hit, \
                   'time', timestamp
            x"""
            #if (multi_hit > 0):
            #    print ('MULTI_HIT!!!')
        if (a[0:2]== '0b10') :
            flagbits= a[0:2].uint
            address = a[2:6].uint        
            ovf = a[6]        
            fast = a[7]
            valley=a[8]
            bgostr= 'BGO, '+str (flagbits)+\
                   ' address, '+str(address)+ \
                   ' ovf, '+str(ovf)+ \
                   ' fast,'+str(fast)
            if ( ovf == 0 ):
                spare=a[9]
                fast_timestamp=a[10:16].uint
                pulse_height=a[16:28].uint
                timestamp=a[28:48].uint

                bgo_det_counts['bgo_address'].append(address)
                bgo_det_counts['bgo_ovf'].append(ovf)
                bgo_det_counts['bgo_fast'].append(fast)
                bgo_det_counts['bgo_valley'].append(valley)
                bgo_det_counts['bgo_pulse_height'].append(pulse_height)
                bgo_det_counts['bgo_timestamp'].append(timestamp)
                bgo_det_counts['bgo_fast_timestamp'].append(fast_timestamp)
                #print bgostr, 'fast_timestamp', fast_timestamp, 'pulse_height', pulse_height,'timestamp', timestamp
            if ( ovf == 1 and fast==0):
                spare=a[9:16].uint
                overflow_duration=a[16:28].uint
                timestamp=a[28:48].uint
                #print bgostr, 'overflow, ', overflow_duration, 'timestamp, ', timestamp
            if ( ovf == 1 and fast==1):
                sample=a[9:16].uint
                adc_value=a[16:28].uint
                timestamp=a[28:48].uint
                #print bgostr, 'sample, ', sample,'adc_value, ', adc_value, 'timestamp', timestamp
    
    return czt_det_counts, bgo_det_counts



def parse_tgf_data(TGFdict):
    trigger_window_thresholds=[int(s) for s in TGFdict['trigger_window_thresholds'].split(' ')]
    trigger_window_durations=[int(s) for s in TGFdict['trigger_window_durations'].split(' ')]
    dau_1_internal_tmons=[int(s) for s in TGFdict['dau_1_internal_tmons'].split(' ')]
    dau_2_internal_tmons=[int(s) for s in TGFdict['dau_2_internal_tmons'].split(' ')]
    dau_3_internal_tmons=[int(s) for s in TGFdict['dau_3_internal_tmons'].split(' ')]
    dau_4_internal_tmons=[int(s) for s in TGFdict['dau_4_internal_tmons'].split(' ')]
    time_of_latest_tcp=[int(s) for s in TGFdict['time_of_latest_tcp'].split(' ')]
    accepted_count_parameters=[int(s) for s in TGFdict['accepted_count_parameters'].split(' ')]
    mmia_triggering_time=[int(s) for s in TGFdict['mmia_triggering_time'].split(' ')]

    mxgs_triggering_count=long(TGFdict['mxgs_triggering_count'],0)


    observation_time=[int(s) for s in TGFdict['observation_time'].split(' ')]


    import bitarray
    ba = bitarray.bitarray()
    ba.fromstring(TGFdict['trigger_window_flags'])
    trigger_window_flags = ba.tolist()

    #print ('mxgs_triggering_count', mxgs_triggering_count)



    #detector_counts=[int(s) for s in TGFdict['detector_counts'].split('\n')]

    detector_counts=TGFdict['detector_counts'].splitlines()


    mytgfobs2=MXGSTGFObservation(
    observation_id                  =int (TGFdict['observation_id'],16),
    utc_year                        =time_of_latest_tcp[0],
    utc_seconds                     =time_of_latest_tcp[1],
    utc_msec                        =time_of_latest_tcp[2],
    tcp_count_dhpu                  =observation_time[0], 
    tcp_count_dpu                   =observation_time[1], 
    dpu_count                       =observation_time[2], 
    dpu_count_prereset              =int (TGFdict['dpu_counter_pre_reset']), 
    dau_bgo_1_int_tmon_chan1        =dau_1_internal_tmons[0], 
    dau_bgo_1_int_tmon_chan2        =dau_1_internal_tmons[1], 
    dau_bgo_1_int_tmon_chan3        =dau_1_internal_tmons[2], 
    dau_bgo_1_int_tmon_chan4        =dau_1_internal_tmons[3],  
    dau_bgo_2_int_tmon_chan1        =dau_2_internal_tmons[0], 
    dau_bgo_2_int_tmon_chan2        =dau_2_internal_tmons[1],
    dau_bgo_2_int_tmon_chan3        =dau_2_internal_tmons[2],
    dau_bgo_2_int_tmon_chan4        =dau_2_internal_tmons[3], 
    dau_bgo_3_int_tmon_chan1        =dau_3_internal_tmons[0], 
    dau_bgo_3_int_tmon_chan2        =dau_3_internal_tmons[1], 
    dau_bgo_3_int_tmon_chan3        =dau_3_internal_tmons[2],  
    dau_bgo_3_int_tmon_chan4        =dau_3_internal_tmons[3], 
    dau_bgo_4_int_tmon_chan1        =dau_4_internal_tmons[0],  
    dau_bgo_4_int_tmon_chan2        =dau_4_internal_tmons[1],  
    dau_bgo_4_int_tmon_chan3        =dau_4_internal_tmons[2],  
    dau_bgo_4_int_tmon_chan4        =dau_4_internal_tmons[3],   
    led_short_win_lr_pulse_height   =accepted_count_parameters[0], 
    led_short_win_upr_pulse_height  =accepted_count_parameters[1], 
    led_long_win_lr_pulse_height    =accepted_count_parameters[2], 
    led_long_win_upr_pulse_height   =accepted_count_parameters[3], 
    hed_short_win_lr_pulse_height   =accepted_count_parameters[4],  
    hed_short_win_upr_pulse_height  =accepted_count_parameters[5], 
    hed_long_win_lr_pulse_height    =accepted_count_parameters[6], 
    hed_long_win_upr_pulse_height   =accepted_count_parameters[7], 
    led_short_win_anticoin_time     =accepted_count_parameters[8], 
    led_long_win_anticoin_time      =accepted_count_parameters[9], 
    hed_short_win_anticoin_time     =accepted_count_parameters[10],  
    hed_long_win_anticoin_time      =accepted_count_parameters[11],  
    led_short_win_flag1             =trigger_window_flags[0],
    led_short_win_flag2             =trigger_window_flags[1],
    led_short_win_flag3             =trigger_window_flags[2],
    led_long_win_flag               =trigger_window_flags[3],
    hed_short_win_flag1             =trigger_window_flags[4],
    hed_short_win_flag2             =trigger_window_flags[5],
    hed_short_win_flag3             =trigger_window_flags[6],
    hed_long_win_flag               =trigger_window_flags[7],
    trig_mmia_enabled               =mmia_triggering_time[0],
    trig_mmia_recd                  =mmia_triggering_time[1],
    led_short_win_dur_1             =trigger_window_durations[0], 
    led_short_win_dur_2             =trigger_window_durations[1], 
    led_short_win_dur_3             =trigger_window_durations[2], 
    led_long_win_dur                =trigger_window_durations[3], 
    hed_short_win_dur_1             =trigger_window_durations[4], 
    hed_short_win_dur_2             =trigger_window_durations[5], 
    hed_short_win_dur_3             =trigger_window_durations[6], 
    hed_long_win_dur                =trigger_window_durations[7], 
    led_short_win_thresh_1          =trigger_window_thresholds[0], 
    led_short_win_thresh_2          =trigger_window_thresholds[1], 
    led_short_win_thresh_3          =trigger_window_thresholds[2], 
    led_long_win_thresh             =trigger_window_thresholds[3], 
    hed_short_win_thresh_1          =trigger_window_thresholds[4], 
    hed_short_win_thresh_2          =trigger_window_thresholds[5], 
    hed_short_win_thresh_3          =trigger_window_thresholds[6], 
    hed_long_win_thresh             =trigger_window_thresholds[7], 
    mxgs_trig_count                 =mxgs_triggering_count,
    mxgs_trig_tcp_count             =8, 
    mxgs_trig_dpu_count             =8, 
    num_counts                      =int (TGFdict['number_of_counts']), 
    detector_counts                 ='asimdata/2016/117/mydata.cdf'  
    )
    mytgfobs2.save()

    czt_det_counts, bgo_det_counts=convert_detector_counts (TGFdict)
    
    debug=0
    if debug==1:
        for key in sorted(mytgfobs2.__dict__.iterkeys()):
            if key not in bad_keys:
                value=mytgfobs2.__dict__[key]
                print (key, value)

    print (mytgfobs2)

    return mytgfobs2, czt_det_counts, bgo_det_counts


def mxgstestdata(fname):
    TGFdict = testfiletodict(fname)

    # read in a file from fname, add the keys to a dict and loop through multiline data fields
    

    mytgfobs2, czt_det_counts, bgo_det_counts=parse_tgf_data(TGFdict)


    
    cdffilename=fname.replace(".txt ",".cdf")
    cdffilename=os.path.basename(fname).replace(".txt",".cdf")
    print (cdffilename)
    writedicttocdf(cdffilename,mytgfobs2,czt_det_counts  , bgo_det_counts)
    print ("done")
    
    return 0
    


path='/Users/garmu/MXGS_PFM_data/PFM_20160513_0912/Section_5_3/TGF/'





fnames=os.listdir(path) 

#fnames=fnamelist.split()
q=MXGSTGFObservation.objects.all().delete()

print fnames
for fname in fnames:    
    mxgstestdata(path+fname)


OperationalError: could not translate host name "db" to address: nodename nor servname provided, or not known


In [3]:


def parseMXGSSampled (fname):
    listofdicts=[]
    sampd={}
    sampbeg={}

    with open (fname) as f:
        for line in f:
            if (line[0] == ' '):
                data=next(f)
                #print (next(f))
                tag2=line.strip()
                tag2=tag2.replace(' ','_')
                tag2=tag2.replace('-','_')
                tag2=tag2.lower()
                sampd[tag2]=data
                #print (line)
            if (line.find('Time of Latest TCP') != -1):
                # add sample to the list
                #print len(listofdicts)print ('how many times does this get called')
                listofdicts.append(sampd)
                if ( len(listofdicts) == 1):
                    dau_1_internal_tmons=[int(s) for s in sampd['dau_1_internal_tmons'].split(' ')]
                    dau_2_internal_tmons=[int(s) for s in sampd['dau_2_internal_tmons'].split(' ')]
                    dau_3_internal_tmons=[int(s) for s in sampd['dau_3_internal_tmons'].split(' ')]
                    dau_4_internal_tmons=[int(s) for s in sampd['dau_4_internal_tmons'].split(' ')]
                    samp=MXGSSampledDetectorCounts(
                        dau_bgo_1_int_tmon_chan1        =dau_1_internal_tmons[0],
                        dau_bgo_1_int_tmon_chan2        =dau_1_internal_tmons[1],
                        dau_bgo_1_int_tmon_chan3        =dau_1_internal_tmons[2],
                        dau_bgo_1_int_tmon_chan4        =dau_1_internal_tmons[3],
                        dau_bgo_2_int_tmon_chan1        =dau_2_internal_tmons[0],
                        dau_bgo_2_int_tmon_chan2        =dau_2_internal_tmons[1],
                        dau_bgo_2_int_tmon_chan3        =dau_2_internal_tmons[2],
                        dau_bgo_2_int_tmon_chan4        =dau_2_internal_tmons[3],
                        dau_bgo_3_int_tmon_chan1        =dau_3_internal_tmons[0],
                        dau_bgo_3_int_tmon_chan2        =dau_3_internal_tmons[1],
                        dau_bgo_3_int_tmon_chan3        =dau_3_internal_tmons[2],
                        dau_bgo_3_int_tmon_chan4        =dau_3_internal_tmons[3],
                        dau_bgo_4_int_tmon_chan1        =dau_4_internal_tmons[0],
                        dau_bgo_4_int_tmon_chan2        =dau_4_internal_tmons[1],
                        dau_bgo_4_int_tmon_chan3        =dau_4_internal_tmons[2],
                        dau_bgo_4_int_tmon_chan4        =dau_4_internal_tmons[3])
                    samp.save()
                if ( len(listofdicts) > 1):
                    timetcp=[int(s) for s in sampd['time_of_latest_tcp'].split(' ')]
                    obstime=[int(s) for s in sampd['observation_time'].split(' ')]
                    data_reduc=[int(s) for s in sampd['dau_data_reduction_factors'].split(' ')]
                    #print sampd
                    samp1sec=MXGSSampledDetectorCounts1Second(
                        mxgs_sampled_detector_counts = samp,
                        utc_year                        =timetcp[0],
                        utc_msec                        =timetcp[1],
                        utc_seconds                     =timetcp[2],
                        tcp_count_dhpu                  =obstime[0],
                        tcp_count_dpu                   =obstime[1],
                        dpu_count_prereset              =int(sampd['dpu_counter_pre_reset']),
                        dpu_count_Sample_Ratio          =data_reduc[0],
                        grey_mode_data_reduc_factor     =data_reduc[1],
                        detector_counts                 =int(sampd['number_of_counts']),
                        detector_events                 ='fixmedirectory/mysamp.cdf')
                    samp1sec.save()
                sampd={}


            if (line.find('Detector Counts') != -1):
                arr=[]
                n=int(sampd['number_of_counts'])
                for i in range (0, n-1):
                    arr.append(next(f))
                sampd['detector_counts']=arr
    return listofdicts



path='/Users/garmu/MXGS_PFM_data/PFM_20160513_0912/Section_5_3/Sampled/'


fnamelist=os.listdir(path) 

print (fnamelist)
#fnamelist="Obs_0007.txt"

#fname=path+fnamelist

q=MXGSSampledDetectorCounts.objects.all().delete()
q=MXGSSampledDetectorCounts1Second.objects.all().delete()
for fname in fnamelist:    
    parseMXGSSampled(path+fname)

['Obs_0007.txt', 'Obs_0009.txt', 'Obs_000b.txt', 'Obs_000e.txt', 'Obs_0011.txt', 'Obs_0015.txt', 'Obs_0017.txt', 'Obs_001a.txt', 'Obs_001d.txt', 'Obs_0020.txt', 'Obs_0023.txt', 'Obs_0026.txt', 'Obs_0029.txt', 'Obs_002c.txt', 'Obs_002e.txt', 'Obs_0032.txt', 'Obs_0036.txt']


NameError: name 'MXGSSampledDetectorCounts' is not defined

In [6]:
def parseMXGSBackground (fname):
    listofdicts=[]
    sampd={}
    sampbeg={}

    with open (fname) as f:
        for line in f:
            if (line[0] == ' ' or line[0] == 'B' ):
                data=next(f)
                #print (next(f))
                tag2=line.strip()
                tag2=tag2.replace(' ','_')
                tag2=tag2.replace('-','_')
                tag2=tag2.lower()
                sampd[tag2]=data
                #print (line)
            if (line.find('Time of Latest TCP') != -1):
                # add sample to the list
                #print len(listofdicts)print ('how many times does this get called')
                listofdicts.append(sampd)
                if ( len(listofdicts) == 1):
                    dau_1_internal_tmons=[int(s) for s in sampd['dau_1_internal_tmons'].split(' ')]
                    dau_2_internal_tmons=[int(s) for s in sampd['dau_2_internal_tmons'].split(' ')]
                    dau_3_internal_tmons=[int(s) for s in sampd['dau_3_internal_tmons'].split(' ')]
                    dau_4_internal_tmons=[int(s) for s in sampd['dau_4_internal_tmons'].split(' ')]
                    accepted_count_parameters=[int(s) for s in sampd['accepted_count_parameters'].split(' ')]
                    led_pulse_height_bin_boundaries=[int(s) for s in sampd['led_pulse_height_bin_boundaries'].split(' ')]
                    hed_pulse_height_bin_boundaries=[int(s) for s in sampd['hed_pulse_height_bin_boundaries'].split(' ')]
                    trigger_window_durations=[int(s) for s in sampd['trigger_window_durations'].split(' ')]
                    samp=MXGSBackgroundObservation(
                        dau_bgo_1_int_tmon_chan1        =dau_1_internal_tmons[0],
                        dau_bgo_1_int_tmon_chan2        =dau_1_internal_tmons[1],
                        dau_bgo_1_int_tmon_chan3        =dau_1_internal_tmons[2],
                        dau_bgo_1_int_tmon_chan4        =dau_1_internal_tmons[3],
                        dau_bgo_2_int_tmon_chan1        =dau_2_internal_tmons[0],
                        dau_bgo_2_int_tmon_chan2        =dau_2_internal_tmons[1],
                        dau_bgo_2_int_tmon_chan3        =dau_2_internal_tmons[2],
                        dau_bgo_2_int_tmon_chan4        =dau_2_internal_tmons[3],
                        dau_bgo_3_int_tmon_chan1        =dau_3_internal_tmons[0],
                        dau_bgo_3_int_tmon_chan2        =dau_3_internal_tmons[1],
                        dau_bgo_3_int_tmon_chan3        =dau_3_internal_tmons[2],
                        dau_bgo_3_int_tmon_chan4        =dau_3_internal_tmons[3],
                        dau_bgo_4_int_tmon_chan1        =dau_4_internal_tmons[0],
                        dau_bgo_4_int_tmon_chan2        =dau_4_internal_tmons[1],
                        dau_bgo_4_int_tmon_chan3        =dau_4_internal_tmons[2],
                        dau_bgo_4_int_tmon_chan4        =dau_4_internal_tmons[3],
                        led_short_win_lr_pulse_height   =accepted_count_parameters[0],
                        led_short_win_upr_pulse_height  =accepted_count_parameters[1],
                        led_long_win_lr_pulse_height    =accepted_count_parameters[2],
                        led_long_win_upr_pulse_height   =accepted_count_parameters[3],
                        hed_short_win_lr_pulse_height   =accepted_count_parameters[4],
                        hed_short_win_upr_pulse_height  =accepted_count_parameters[5],
                        hed_long_win_lr_pulse_height    =accepted_count_parameters[6],
                        hed_long_win_upr_pulse_height   =accepted_count_parameters[7],
                        led_short_win_anticoin_time     =accepted_count_parameters[8],
                        led_long_win_anticoin_time      =accepted_count_parameters[9],
                        hed_short_win_anticoin_time     =accepted_count_parameters[10],
                        hed_long_win_anticoin_time      =accepted_count_parameters[11],
                        led_bin0_lr_boundary            =led_pulse_height_bin_boundaries[0],
                        led_bin1_lr_boundary            =led_pulse_height_bin_boundaries[1],
                        led_bin2_lr_boundary            =led_pulse_height_bin_boundaries[2],
                        led_bin3_lr_boundary            =led_pulse_height_bin_boundaries[3],
                        led_bin4_lr_boundary            =led_pulse_height_bin_boundaries[4],
                        led_bin5_lr_boundary            =led_pulse_height_bin_boundaries[5],
                        led_bin6_lr_boundary            =led_pulse_height_bin_boundaries[6],
                        led_bin7_lr_boundary            =led_pulse_height_bin_boundaries[7],
                        led_bin8_lr_boundary            =led_pulse_height_bin_boundaries[8],
                        led_bin9_lr_boundary            =led_pulse_height_bin_boundaries[9],
                        led_bin9_upr_boundary           =led_pulse_height_bin_boundaries[10],
                        hed_bin0_lr_boundary            =hed_pulse_height_bin_boundaries[0],
                        hed_bin1_lr_boundary            =hed_pulse_height_bin_boundaries[1],
                        hed_bin2_lr_boundary            =hed_pulse_height_bin_boundaries[2],
                        hed_bin3_lr_boundary            =hed_pulse_height_bin_boundaries[3],
                        hed_bin4_lr_boundary            =hed_pulse_height_bin_boundaries[4],
                        hed_bin5_lr_boundary            =hed_pulse_height_bin_boundaries[5],
                        hed_bin6_lr_boundary            =hed_pulse_height_bin_boundaries[6],
                        hed_bin7_lr_boundary            =hed_pulse_height_bin_boundaries[7],
                        hed_bin8_lr_boundary            =hed_pulse_height_bin_boundaries[8],
                        hed_bin9_lr_boundary            =hed_pulse_height_bin_boundaries[9],
                        hed_bin9_upr_boundary           =hed_pulse_height_bin_boundaries[10],
                        led_short_win_dur_1             =trigger_window_durations[0],
                        led_short_win_dur_2             =trigger_window_durations[1],
                        led_short_win_dur_3             =trigger_window_durations[2],
                        led_long_win_dur                =trigger_window_durations[3],
                        hed_short_win_dur_1             =trigger_window_durations[4],
                        hed_short_win_dur_2             =trigger_window_durations[5],
                        hed_short_win_dur_3             =trigger_window_durations[6],
                        hed_long_win_dur                =trigger_window_durations[7])
                    samp.save()
                if ( len(listofdicts) > 1):
                    timetcp=[int(s) for s in sampd['time_of_latest_tcp'].split(' ')]
                    obstime=[int(s) for s in sampd['observation_time'].split(' ')]
                    total_count_ratemeters=[int(s) for s in sampd['total_count_ratemeters'].split(' ')]
                    accepted_count_ratemeters=[int(s) for s in sampd['accepted_count_ratemeters'].split(' ')]
                    calculated_background_rates=[int(s) for s in sampd['calculated_background_rates'].split(' ')]
                    trigger_window_thresholds=[int(s) for s in sampd['trigger_window_thresholds'].split(' ')]
                    czt_daus_current_offset_values=[int(s) for s in sampd['czt_daus_current_offset_values'].split(' ')]
                    bgo_daus_current_offset_values=[int(s) for s in sampd['bgo_daus_current_offset_values'].split(' ')]
#    'bgo_daus_current_offset_values': '249 120 186 138 103 143 12 193 47 167 90 73 453 230 129 204\n',
#    'detector_counts': [   '47 65 53 40 45 45 53 57 72 57 35 54 48 61 62 60 53 51 39 48 62 66 40 51 43 56 62 60 51 48 24\n'],
#    'pulse_height_binned_data': '0 82 241 234 162 95 70 67 40 0\n',
#    'time_binned_data': '29 42 35 30 29 37 31 36 34 35 20 24 32 34 29 37 38 39 25 31 30 36 31 28 29 41 35 44 23 30 17\n',
                    samp1sec=MXGSBackgroundObservation1Second(
                        background_observation= samp,
                        utc_year                        =timetcp[0],
                        utc_msec                        =timetcp[1],
                        utc_seconds                     =timetcp[2],
                        tcp_count_dhpu                  =obstime[0],
                        tcp_count_dpu                   =obstime[1],
                        dpu_count_prereset              =int(sampd['dpu_counter_pre_reset']),
                        sw_mode                         ='UD',
                        sw_submode                      ='UD',
                        dau_data_reduc_factor           =int(sampd['dau_data_reduction_factor']),
                        led_count_ratemeter             =total_count_ratemeters[0],
                        hed_count_ratemeter             =total_count_ratemeters[1],
                        dau_total_rate                  =int(sampd['dau_total_rate']),
                        led_accept_count_rate_short     =accepted_count_ratemeters[0],
                        led_accept_count_rate_long      =accepted_count_ratemeters[1],
                        hed_accept_count_rate_short     =accepted_count_ratemeters[2],
                        hed_accept_count_rate_long      =accepted_count_ratemeters[3],
                        led_calc_background_rate_short  =calculated_background_rates[0],
                        led_calc_background_rate_long   =calculated_background_rates[1],
                        hed_calc_background_rate_short  =calculated_background_rates[2],
                        hed_calc_background_rate_long   =calculated_background_rates[3],
                        led_short_win_thresh_1          =trigger_window_thresholds[0],
                        led_short_win_thresh_2          =trigger_window_thresholds[1],
                        led_short_win_thresh_3          =trigger_window_thresholds[2],
                        led_long_win_thresh             =trigger_window_thresholds[3],
                        hed_short_win_thresh_1          =trigger_window_thresholds[4],
                        hed_short_win_thresh_2          =trigger_window_thresholds[5],
                        hed_short_win_thresh_3          =trigger_window_thresholds[6],
                        hed_long_win_thresh             =trigger_window_thresholds[7],
                        dau1_dm_if_1_current_offset     =czt_daus_current_offset_values[0],
                        dau1_dm_if_2_current_offset     =czt_daus_current_offset_values[1],
                        dau1_dm_if_3_current_offset     =czt_daus_current_offset_values[2],
                        dau1_dm_if_4_current_offset     =czt_daus_current_offset_values[3],
                        dau2_dm_if_1_current_offset     =czt_daus_current_offset_values[4],
                        dau2_dm_if_2_current_offset     =czt_daus_current_offset_values[5],
                        dau2_dm_if_3_current_offset     =czt_daus_current_offset_values[6],
                        dau2_dm_if_4_current_offset     =czt_daus_current_offset_values[7],
                        dau3_dm_if_1_current_offset     =czt_daus_current_offset_values[8],
                        dau3_dm_if_2_current_offset     =czt_daus_current_offset_values[9],
                        dau3_dm_if_3_current_offset     =czt_daus_current_offset_values[10],
                        dau3_dm_if_4_current_offset     =czt_daus_current_offset_values[11],
                        dau4_dm_if_1_current_offset     =czt_daus_current_offset_values[12],
                        dau4_dm_if_2_current_offset     =czt_daus_current_offset_values[13],
                        dau4_dm_if_3_current_offset     =czt_daus_current_offset_values[14],
                        dau4_dm_if_4_current_offset     =czt_daus_current_offset_values[15],
                        dau1_pmt_if_1_current_offset    =bgo_daus_current_offset_values[0],
                        dau1_pmt_if_2_current_offset    =bgo_daus_current_offset_values[1],
                        dau1_pmt_if_3_current_offset    =bgo_daus_current_offset_values[2],
                        dau2_pmt_if_1_current_offset    =bgo_daus_current_offset_values[3],
                        dau2_pmt_if_2_current_offset    =bgo_daus_current_offset_values[4],
                        dau2_pmt_if_3_current_offset    =bgo_daus_current_offset_values[5],
                        dau3_pmt_if_1_current_offset    =bgo_daus_current_offset_values[6],
                        dau3_pmt_if_2_current_offset    =bgo_daus_current_offset_values[7],
                        dau3_pmt_if_3_current_offset    =bgo_daus_current_offset_values[8],
                        dau4_pmt_if_1_current_offset    =bgo_daus_current_offset_values[9],
                        dau4_pmt_if_2_current_offset    =bgo_daus_current_offset_values[10],
                        dau4_pmt_if_3_current_offset    =bgo_daus_current_offset_values[11]            
                        #led_pulse_height_bin            =[0,0,0,0,0,0,0,0,0,0],
                        #hed_pulse_height_bin            =[0,0,0,0,0,0,0,0,0,0],
                        #led_time_bin_values             =[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                        #hed_time_bin_values             =[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                    )
                    samp1sec.save()
                sampd={}


            if (line.find('Pulse-Height Binned Data') != -1):
                arr=[]
                n=2
                for i in range (0, n-1):
                    arr.append(next(f))
                sampd['detector_counts']=arr
            if (line.find('Time Binned Data') != -1):
                arr=[]
                n=2
                for i in range (0, n-1):
                    arr.append(next(f))
                sampd['detector_counts']=arr
    return listofdicts



path='/Users/garmu/MXGS_PFM_data/PFM_20160513_0912/Section_5_3/Background/'


fnamelist=os.listdir(path) 

print (fnamelist)
#fnamelist="Obs_0007.txt"

#fname=path+fnamelist

q=MXGSBackgroundObservation.objects.all().delete()
q=MXGSBackgroundObservation1Second.objects.all().delete()
for fname in fnamelist:    
    a=parseMXGSBackground(path+fname)

['Obs_0006.txt', 'Obs_0008.txt', 'Obs_000a.txt', 'Obs_000d.txt', 'Obs_000f.txt', 'Obs_0012.txt', 'Obs_0016.txt', 'Obs_0019.txt', 'Obs_001c.txt', 'Obs_001f.txt', 'Obs_0021.txt', 'Obs_0025.txt', 'Obs_0028.txt', 'Obs_002b.txt', 'Obs_002d.txt', 'Obs_0030.txt', 'Obs_0034.txt']


NameError: name 'MXGSBackgroundObservation' is not defined

In [7]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(a[1])

NameError: name 'a' is not defined

In [8]:
path='/Users/garmu/MXGS_PFM_data/PFM_20160513_0912/Section_5_3/raw/'


fname='PFM_20160513_0912_raw.txt'

with open (path+fname) as f:
    for line in f:
        if "809202a002b70005" in line:
            #print line
            data=line
            
samp={}

import textwrap
dl=textwrap.wrap(data, 4)

x=iter(dl)
samp['header']=x.next()+x.next()
samp['fixme1']=x.next()
samp['observation_id']=int (x.next(),16)
samp['fixme2']        =int (x.next(),16)
samp['fixme3']        =int (x.next(),16)

samp['dau_bgo_1_int_tmon_chan1']        =int (x.next(),16)
samp['dau_bgo_1_int_tmon_chan2']        =int (x.next(),16)
samp['dau_bgo_1_int_tmon_chan3']        =int (x.next(),16)
samp['dau_bgo_1_int_tmon_chan4']        =int (x.next(),16)
samp['dau_bgo_2_int_tmon_chan1']        =int (x.next(),16)
samp['dau_bgo_2_int_tmon_chan2']        =int (x.next(),16)
samp['dau_bgo_2_int_tmon_chan3']        =int (x.next(),16)
samp['dau_bgo_2_int_tmon_chan4']        =int (x.next(),16)
samp['dau_bgo_3_int_tmon_chan1']        =int (x.next(),16)
samp['dau_bgo_3_int_tmon_chan2']        =int (x.next(),16)
samp['dau_bgo_3_int_tmon_chan3']        =int (x.next(),16)
samp['dau_bgo_3_int_tmon_chan4']        =int (x.next(),16)
samp['dau_bgo_4_int_tmon_chan1']        =int (x.next(),16)
samp['dau_bgo_4_int_tmon_chan2']        =int (x.next(),16)
samp['dau_bgo_4_int_tmon_chan3']        =int (x.next(),16)
samp['dau_bgo_4_int_tmon_chan4']        =int (x.next(),16)

bad_keys=['mybadkey']
for key in sorted(samp.iterkeys()):
    if key not in bad_keys:
        value=samp[key]
        print (key, value)
listofdicts=[]

n1=60-58

samp1sec={}

samp1sec['utc_year ']                       =int(x.next(),16)
samp1sec['utc_msec']                        =int(x.next(),16)

samp1sec['utc_seconds']                     =int (x.next()+x.next(),16)
samp1sec['tcp_count_dhpu']                  =int (x.next()+x.next(),16)
samp1sec['tcp_count_dpu']                   =int (x.next()+x.next(),16)
samp1sec['dpu_count_prereset']              =int (x.next()+x.next(),16)
a=x.next()
#samp1sec['dpu_count_Sample_Ratio']          =int (a[0:2],16)
#samp1sec['grey_mode_data_reduc_factor']     =int (a[2:4],16)
samp1sec['detector_counts']                 =int (x.next()+x.next(),16)

arr=[]
#for i in range(0,samp1sec['detector_counts']):

#getridofme=x.next()
#arr.append(  x.next()+x.next()+x.next()  )
samp1sec['detector_events']=arr



print ('=====')

bad_keys=['mybadkey']
for key in sorted(samp1sec.iterkeys()):
    if key not in bad_keys:
        value=samp1sec[key]
        print (key, value)
        
        
listofdicts=[]

    
import itertools
det_ev=list(itertools.islice(x, 0, samp1sec['detector_counts']  , 1))   
arr=[i+j+k+l for i,j,k,l in zip(det_ev[::4], det_ev[1::4] , det_ev[2::4], det_ev[3::4])]

    
#print det_ev

#print (data)

('dau_bgo_1_int_tmon_chan1', 1970)
('dau_bgo_1_int_tmon_chan2', 0)
('dau_bgo_1_int_tmon_chan3', 0)
('dau_bgo_1_int_tmon_chan4', 1016)
('dau_bgo_2_int_tmon_chan1', 0)
('dau_bgo_2_int_tmon_chan2', 1016)
('dau_bgo_2_int_tmon_chan3', 0)
('dau_bgo_2_int_tmon_chan4', 1015)
('dau_bgo_3_int_tmon_chan1', 10)
('dau_bgo_3_int_tmon_chan2', 26127)
('dau_bgo_3_int_tmon_chan3', 15)
('dau_bgo_3_int_tmon_chan4', 16980)
('dau_bgo_4_int_tmon_chan1', 1127)
('dau_bgo_4_int_tmon_chan2', 1111)
('dau_bgo_4_int_tmon_chan3', 1111)
('dau_bgo_4_int_tmon_chan4', 3120)
('fixme1', '02b7')
('fixme2', 0)
('fixme3', 1384)
('header', '809202a0')
('observation_id', 5)
=====
('detector_counts', 204210376)
('detector_events', [])
('dpu_count_prereset', 73663570)
('tcp_count_dhpu', 74253407)
('tcp_count_dpu', 73141310)
('utc_msec', 1109)
('utc_seconds', 72551468)
('utc_year ', 1123)


In [9]:
path='/Users/garmu/MXGS_PFM_data/PFM_20160513_0912/Section_5_3/raw/'


fname='PFM_20160513_0912_raw.txt'
i=0
import re

regex = re.compile('809202a[01]....0005')

datalist=[]

masterdata=[]
total=0
with open (path+fname) as f:    
    for line in f:
        if re.match(regex,line):
            #print line
            datalist.append(line)
            a=datalist[i]
            #convert to list here
            import textwrap
            dl=textwrap.wrap(a, 4)
            
            total =  total + int (a[8:12],16)
            #print i, a[0:8], int (a[8:12],16), a[12:16], a[16:20], a[20:24], a[24:28]
            if ( i > 0) :
            # delete first 14 bytes (Continuation Header)
                dl=dl[28:]
            i=i+1
            #if i == 0 :
            # delete first 5 bytes (Continuation Header)
            #    dl=dl[10:]
            masterdata.extend(dl)
            
samp={}

import textwrap
#dl=textwrap.wrap(data, 4)


x=iter(masterdata)
samp['header']=x.next()+x.next()

print (samp)
samp['packet_length']=int (x.next(),16)
samp['observation_id']=int (x.next(),16)
samp['continuation_id']        =int (x.next(),16)
samp['last_packet']        =int (x.next(),16)
samp['utc_year ']                       =int(x.next(),16)
samp['utc_msec']                        =int(x.next(),16)

samp['utc_seconds']                     =int (x.next()+x.next(),16)
samp['tcp_count_dhpu']                  =int (x.next()+x.next(),16)
samp['tcp_count_dpu']                   =int (x.next()+x.next(),16)
samp['dpu_counter_current']                   =int (x.next()+x.next(),16)

samp['dpu_count_prereset']              =int (x.next()+x.next(),16)
samp['dau_bgo_1_int_tmon_chan1']        =int (x.next(),16)
samp['dau_bgo_1_int_tmon_chan2']        =int (x.next(),16)
samp['dau_bgo_1_int_tmon_chan3']        =int (x.next(),16)
samp['dau_bgo_1_int_tmon_chan4']        =int (x.next(),16)
samp['dau_bgo_2_int_tmon_chan1']        =int (x.next(),16)
samp['dau_bgo_2_int_tmon_chan2']        =int (x.next(),16)
samp['dau_bgo_2_int_tmon_chan3']        =int (x.next(),16)
samp['dau_bgo_2_int_tmon_chan4']        =int (x.next(),16)
samp['dau_bgo_3_int_tmon_chan1']        =int (x.next(),16)
samp['dau_bgo_3_int_tmon_chan2']        =int (x.next(),16)
samp['dau_bgo_3_int_tmon_chan3']        =int (x.next(),16)
samp['dau_bgo_3_int_tmon_chan4']        =int (x.next(),16)
samp['dau_bgo_4_int_tmon_chan1']        =int (x.next(),16)
samp['dau_bgo_4_int_tmon_chan2']        =int (x.next(),16)
samp['dau_bgo_4_int_tmon_chan3']        =int (x.next(),16)
samp['dau_bgo_4_int_tmon_chan4']        =int (x.next(),16)

samp ['led_short_win_lr_pulse_height'] = int (x.next(),16) 
samp ['led_short_win_upr_pulse_height'] = int (x.next(),16) 
samp ['led_long_win_lr_pulse_height'] = int (x.next(),16) 
samp ['led_long_win_upr_pulse_height'] = int (x.next(),16) 
samp ['hed_short_win_lr_pulse_height'] = int (x.next(),16) 
samp ['hed_short_win_upr_pulse_height'] = int (x.next(),16) 
samp ['hed_long_win_lr_pulse_height'] = int (x.next(),16) 
samp ['hed_long_win_upr_pulse_height'] = int (x.next(),16) 


shortlongwin=x.next()
samp ['led_short_win_anticoin_time'] =int (shortlongwin[0:2],16) 
samp ['led_long_win_anticoin_time'] = int (shortlongwin[2:4],16) 
shortlongwin=x.next()
samp ['hed_short_win_anticoin_time'] = int (shortlongwin[0:2],16) 
samp ['hed_long_win_anticoin_time'] = int (shortlongwin[2:4],16)

bits=x.next()

from bitstring import BitArray

a = BitArray('0x'+bits)
samp ['led_short_win_flag1'] = a[0]
samp ['led_short_win_flag2'] = a[1]
samp ['led_short_win_flag3'] = a[2]
samp ['led_long_win_flag'] = a[3]
samp ['hed_short_win_flag1'] = a[4]
samp ['hed_short_win_flag2'] = a[5]
samp ['hed_short_win_flag3'] = a[6]
samp ['hed_long_win_flag'] =a[7]

print bits

samp ['trig_mmia_enabled'] = 1
samp ['trig_mmia_recd'] = 1


samp ['led_short_win_dur_1'] =  int (x.next(),16) 
samp ['led_short_win_dur_2'] = int (x.next(),16) 
samp ['led_short_win_dur_3'] = int (x.next(),16) 
samp ['led_long_win_dur'] = int (x.next(),16) 
samp ['hed_short_win_dur_1'] = int (x.next(),16) 
samp ['hed_short_win_dur_2'] = int (x.next(),16) 
samp ['hed_short_win_dur_3'] = int (x.next(),16) 
samp ['hed_long_win_dur'] = int (x.next(),16) 




scrh=x.next()

samp ['led_short_win_thresh_1'] = int (scrh[0:2],16)
samp ['led_short_win_thresh_2'] = int (scrh[2:4],16) 

scrh1=x.next()
scrh2=x.next()
scrh3=x.next()

samp ['led_short_win_thresh_3'] = int (scrh1[0:2],16) 
samp ['led_long_win_thresh']    = int (scrh1[2:4]+scrh2[0:2],16) 
samp ['hed_short_win_thresh_1'] = int (scrh2[2:4],16) 
samp ['hed_short_win_thresh_2'] = int (scrh3[0:2],16) 
samp ['hed_short_win_thresh_3'] = int (scrh3[2:4],16) 
samp ['hed_long_win_thresh'] = int (x.next(),16) 



samp ['mxgs_trig_count'] = x.next()+x.next()+x.next()+x.next() 
samp ['mxgs_trig_tcp_count'] = int (x.next()+x.next(),16) 
samp ['mxgs_trig_dpu_count'] = int (x.next()+x.next(),16) 
samp ['num_counts'] = int (x.next()+x.next(),16)


#for i in range (0,samp ['num_counts']-1):
#    print i, x.next()+x.next()+x.next()+x.next()

import itertools
det_ev=list(itertools.islice(x,  None  ))   
arr=[i+j+k+l for i,j,k,l in zip(det_ev[::4], det_ev[1::4] , det_ev[2::4], det_ev[3::4])]

bad_keys=['mybadkey']
for key in sorted(samp.iterkeys()):
    if key not in bad_keys:
        value=samp[key]
        print (key, value)

{'header': '809202a0'}
0180
('continuation_id', 0)
('dau_bgo_1_int_tmon_chan1', 1127)
('dau_bgo_1_int_tmon_chan2', 1111)
('dau_bgo_1_int_tmon_chan3', 1111)
('dau_bgo_1_int_tmon_chan4', 3120)
('dau_bgo_2_int_tmon_chan1', 1123)
('dau_bgo_2_int_tmon_chan2', 1109)
('dau_bgo_2_int_tmon_chan3', 1107)
('dau_bgo_2_int_tmon_chan4', 3116)
('dau_bgo_3_int_tmon_chan1', 1133)
('dau_bgo_3_int_tmon_chan2', 1119)
('dau_bgo_3_int_tmon_chan3', 1116)
('dau_bgo_3_int_tmon_chan4', 3134)
('dau_bgo_4_int_tmon_chan1', 1124)
('dau_bgo_4_int_tmon_chan2', 1106)
('dau_bgo_4_int_tmon_chan3', 1107)
('dau_bgo_4_int_tmon_chan4', 3116)
('dpu_count_prereset', 1000020)
('dpu_counter_current', 681487)
('header', '809202a0')
('hed_long_win_anticoin_time', 4)
('hed_long_win_dur', 25000)
('hed_long_win_flag', True)
('hed_long_win_lr_pulse_height', 16)
('hed_long_win_thresh', 24)
('hed_long_win_upr_pulse_height', 1023)
('hed_short_win_anticoin_time', 4)
('hed_short_win_dur_1', 300)
('hed_short_win_dur_2', 1000)
('hed_short_w

In [10]:
path='/Users/garmu/MXGS_PFM_data/PFM_20160513_0912/Section_5_3/raw/'


fname='PFM_20160513_0912_raw.txt'
i=0
import re

mxgs='80'
ground='92'
sciencedata='02'

regex = re.compile(mxgs+ground+sciencedata+'a.....0005')
regex = re.compile(mxgs+ground+sciencedata+'a[01]')

datalist=[]

masterdata=[]

allobsids=[]
total=0
# first pass: determine all unique ids
with open (path+fname) as f:    
    for line in f:
        if re.match(regex,line):
            #print line
            datalist.append(line)
            a=datalist[i]
            #convert to list here
            import textwrap
            #dl=textwrap.wrap(a, 4)
            obsid=a[12:16]
            allobsids.append(obsid)
            total =  total + int (a[8:12],16)
            #print i, a[0:8], int (a[8:12],16), a[12:16], a[16:20], a[20:24], a[24:28]
            #print i
            if ( i > 0) :
            # delete first 14 bytes (Continuation Header)
                dl=dl[28:]
            i=i+1
            #if i == 0 :
            # delete first 5 bytes (Continuation Header)
            #    dl=dl[10:]
            #masterdata.append(dl)
            

uniqueids= set(allobsids)


#create a dict with all observations

print uniqueids

dictoflists={}


for uniqueid in uniqueids:
    regex = re.compile(mxgs+ground+sciencedata+'a[01]....'+uniqueid)
    with open (path+fname) as f: 
        obslist=[]
        for line in f:
            if re.match(regex,line):
                obslist.append(line)
        dictoflists[uniqueid]=obslist
        # need to use the continuity id to sort
        # sort by two fields, the head/continuity field first and then the continuity id
        
        
def TGFtoDjango():
    pass
    return 0

# trim the first few off the list and hand them over to the main data packet
# 

for key in sorted(dictoflists.iterkeys()):
    val=dictoflists[key]
    print len(val)
    # do the sorting here
    mystr=val[0]
    print key, mystr[0:32]
    # func that does the convertion to Django class
    if 'a0' == 'a0':
        TGFtoDjango()

#print (total/4)


set(['0005', '0027', '0024', '0018', '001e', '002a', '002f', '001b', '000c'])
51
0005 809202a002b700050000056807b20000
43
000c 809202a002b7000c0000056807b20000
47
0018 809202a002c500180000058407b20000
44
001b 809202a002c1001b0000057c07b20000
42
001e 809202a002c7001e0000058807b20000
47
0024 809202a002c700240000058807b20000
47
0027 809202a002bb00270000057007b20000
46
002a 809202a002cb002a0000059007b20000
42
002f 809202a002bd002f0000057407b20000


In [11]:
for key in sorted(dictoflists.iterkeys()):
    val=dictoflists[key]
    mystr=val[0]
    print key, mystr[0:32], 'MXGS', 'TGF', 'obsid=', mystr[12:16]


0005 809202a002b700050000056807b20000 MXGS TGF obsid= 0005
000c 809202a002b7000c0000056807b20000 MXGS TGF obsid= 000c
0018 809202a002c500180000058407b20000 MXGS TGF obsid= 0018
001b 809202a002c1001b0000057c07b20000 MXGS TGF obsid= 001b
001e 809202a002c7001e0000058807b20000 MXGS TGF obsid= 001e
0024 809202a002c700240000058807b20000 MXGS TGF obsid= 0024
0027 809202a002bb00270000057007b20000 MXGS TGF obsid= 0027
002a 809202a002cb002a0000059007b20000 MXGS TGF obsid= 002a
002f 809202a002bd002f0000057407b20000 MXGS TGF obsid= 002f
